### Import packages

In [2]:
import pandas as pd
import gensim
from gensim.utils import simple_preprocess
import os
import pyLDAvis
import pyLDAvis.gensim as gensimvis
import numpy as np
import tqdm
from tqdm import tqdm
import re
from pprint import pprint
import nltk
from nltk.corpus import stopwords
import spacy
import gensim.corpora as corpora
from pprint import pprint
from gensim.models import CoherenceModel
pd.set_option('display.max_rows', None)

C:\Users\15138\anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [3]:
bbe = pd.read_csv('t_bbe.csv')

In [4]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

In [5]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [6]:
# supporting function
def compute_coherence_values(corpus, dictionary, k, a, b):
    lda_model = gensim.models.LdaModel(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=k,
                                           chunksize=100,
                                           passes=10,
                                           alpha=a,
                                           eta=b)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

In [7]:
def _get_best_num_topics(topics_range, alpha_lst, beta_lst, corpus):
    corpus_sets = [corpus]
    # maximum coherence score (value)
    max_cv = 0.0
    alpha = 0.0
    beta = 0.0
    num_topics = 2

    # iterate through the corus
    for i in tqdm(range(len(corpus_sets))):
        # iterate through number of topics
        for k in topics_range:
            # iterate through alpha values
            for a in alpha_lst:
                # iterare through beta values
                for b in beta_lst:
                    # compute coherence value
                    cv = compute_coherence_values(corpus=corpus_sets[i], 
                                                dictionary=id2word, k=k, a=a, b=b)
                    if cv > max_cv:
                        max_cv = cv
                        alpha = a
                        beta = b
                        num_topics = k

    return alpha, beta, num_topics, max_cv

# BBE

## Books

### Preprocessing

In [16]:
# Create list of book documents and put them in a dataframe
booksList = []
for book in bbe.groupby('b')['t']:
    booksList.append(' '.join(book[1].tolist()))
books = pd.DataFrame(booksList, columns =['t'])

In [17]:
# Remove punctuation
books['t_processed'] = books['t'].map(lambda x: re.sub('[,\.!?]', '', x))

# Convert the titles to lowercase
books['t_processed'] = books['t_processed'].map(lambda x: x.lower())

#### Tokenize words and further clean-up text

data = books.t_processed.values.tolist()
data_words = list(sent_to_words(data))

#### Creating Bigram and Trigram Models

# Build the bigram and trigram models

bigram = gensim.models.Phrases(data_words, min_count=3, threshold=5)
trigram = gensim.models.Phrases(bigram[data_words], threshold=5)  
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

#### Remove Stopwords, Make Bigrams and Lemmatize

#list of stopwords
with open('stop_words_lst.txt') as f:
    standard_stop_words = f.read().splitlines()
with open('bible_stop_words_lst.txt') as f:
    bible_stop_words = f.read().splitlines()
stop_words = standard_stop_words + bible_stop_words
### Call the functions above

# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
nlp = spacy.load("en_core_web_lg", disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

### Data transformation: Corpus and Dictionary

# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

<>:2: DeprecationWarning: invalid escape sequence \.
<>:2: DeprecationWarning: invalid escape sequence \.
C:\Users\15138\AppData\Local\Temp/ipykernel_13700/1686976997.py:2: DeprecationWarning: invalid escape sequence \.
  books['t_processed'] = books['t'].map(lambda x: re.sub('[,\.!?]', '', x))


### Model Selection

In [10]:
# Topics range
min_topics = 2
max_topics = 66
step_size = 2
topics_range = range(min_topics, max_topics, step_size)

# List of alpha parameters
alpha_lst = ['auto']

# List of beta parameters
beta_lst = list(np.arange(0.01, 3, 0.1))

# Call to the method above
alpha, beta, num_topics, max_cv = _get_best_num_topics(topics_range, alpha_lst, beta_lst, corpus)

print("alpha: ", alpha)
print("beta: ", beta)
print("Number of topics: ", num_topics)
print("Optimal Coherence Value: ", max_cv)

### Final Model Training

lda_model = gensim.models.LdaModel(corpus=corpus,
                                      id2word=id2word,
                                      num_topics=num_topics,
                                      chunksize=100,
                                      passes=10,
                                      alpha=alpha,
                                      eta=beta)

# Print the Keyword in the 10 topics
pprint(lda_model.print_topics(num_topics=-1, num_words=20))
doc_lda = lda_model[corpus]

# Visualize the topics
pyLDAvis.enable_notebook()
LDAvis_prepared = gensimvis.prepare(lda_model, corpus, id2word)
LDAvis_prepared

100%|██████████| 1/1 [4:25:40<00:00, 15940.30s/it]


alpha:  auto
beta:  1.81
Number of topics:  8
Optimal Coherence Value:  0.4455170228192671
[(0,
  '0.011*"place" + 0.010*"evil" + 0.009*"death" + 0.009*"servant" + '
  '0.008*"fear" + 0.008*"heart" + 0.008*"earth" + 0.008*"king" + 0.008*"town" '
  '+ 0.007*"live" + 0.007*"time" + 0.007*"word" + 0.007*"food" + 0.007*"order" '
  '+ 0.007*"turn" + 0.007*"ear" + 0.007*"eye" + 0.007*"hand" + 0.006*"father" '
  '+ 0.006*"work"'),
 (1,
  '0.017*"family" + 0.013*"tent" + 0.011*"number" + 0.010*"levite" + '
  '0.010*"order" + 0.010*"offer" + 0.007*"priest" + 0.007*"work" + '
  '0.006*"tribe" + 0.006*"place" + 0.006*"meeting" + 0.006*"gold" + '
  '0.005*"town" + 0.005*"death" + 0.005*"chief" + 0.005*"lamb" + 0.005*"year" '
  '+ 0.004*"altar" + 0.004*"meal_offere" + 0.004*"vessel"'),
 (2,
  '0.001*"death" + 0.001*"place" + 0.001*"servant" + 0.000*"earth" + '
  '0.000*"evil" + 0.000*"ear" + 0.000*"food" + 0.000*"heart" + 0.000*"work" + '
  '0.000*"order" + 0.000*"cover" + 0.000*"brother" + 0.000*"

C:\Users\15138\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:228: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info  = pd.DataFrame({'saliency': saliency, 'Term': vocab, \


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.219171  0.007339       1        1  84.140713
1      0.028447 -0.083477       2        1   7.884228
6      0.025468  0.099376       3        1   4.850615
4      0.013328 -0.040871       4        1   3.119328
3     -0.060193  0.007196       5        1   0.001454
2     -0.072681  0.003248       6        1   0.001278
5     -0.074799  0.003865       7        1   0.001264
7     -0.078739  0.003324       8        1   0.001119, topic_info=                  Term         Freq        Total Category  logprob  loglift
243               food   876.000000   876.000000  Default  30.0000  30.0000
347          knowledge   726.000000   726.000000  Default  29.0000  29.0000
198               evil  1159.000000  1159.000000  Default  28.0000  28.0000
478              place  1353.000000  1353.000000  Default  27.0000  27.0000
209              faith   508.000000   508.000000  Default  26.0000  26.0000
144              death  1170.000000  1170.000000  Default  25.0000  25.0000
448              order   963.000000   963.000000  Default  24.0000  24.0000
754               word   879.000000   879.000000  Default  23.0000  23.0000
755               work   846.000000   846.000000  Default  22.0000  22.0000
176              earth   911.000000   911.000000  Default  21.0000  21.0000
299              heart   948.000000   948.000000  Default  20.0000  20.0000
494             priest   538.000000   538.000000  Default  19.0000  19.0000
217               fear   945.000000   945.000000  Default  18.0000  18.0000
365               live   920.000000   920.000000  Default  17.0000  17.0000
590                sin   675.000000   675.000000  Default  16.0000  16.0000
173                ear   798.000000   798.000000  Default  15.0000  15.0000
567            servant  1096.000000  1096.000000  Default  14.0000  14.0000
150             desire   513.000000   513.000000  Default  13.0000  13.0000
684               turn   813.000000   813.000000  Default  12.0000  12.0000
674               town   932.000000   932.000000  Default  11.0000  11.0000
343               king   904.000000   904.000000  Default  10.0000  10.0000
371               love   415.000000   415.000000  Default   9.0000   9.0000
665               time   894.000000   894.000000  Default   8.0000   8.0000
289               hand   806.000000   806.000000  Default   7.0000   7.0000
683               true   444.000000   444.000000  Default   6.0000   6.0000
14              answer   614.000000   614.000000  Default   5.0000   5.0000
431             number   613.000000   613.000000  Default   4.0000   4.0000
360               life   660.000000   660.000000  Default   3.0000   3.0000
203                eye   803.000000   803.000000  Default   2.0000   2.0000
71             brother   510.000000   510.000000  Default   1.0000   1.0000
14              answer   607.363068   614.471002   Topic1  -5.2404   0.1610
711              voice   361.692665   366.672616   Topic1  -5.7588   0.1590
702            upright   346.691841   352.099488   Topic1  -5.8011   0.1572
2729          disciple   252.079724   256.012160   Topic1  -6.1198   0.1572
553               seat   432.140426   439.639917   Topic1  -5.5808   0.1555
343               king   889.347735   904.862930   Topic1  -4.8591   0.1554
173                ear   784.469638   798.438497   Topic1  -4.9845   0.1550
129                cry   448.486565   457.195759   Topic1  -5.5437   0.1534
730                way   255.259775   260.326215   Topic1  -6.1073   0.1530
176              earth   891.930318   911.064315   Topic1  -4.8562   0.1515
217               fear   925.119436   945.184876   Topic1  -4.8196   0.1512
420             nation   518.036631   529.419771   Topic1  -5.3995   0.1509
612               soul   297.639678   304.480647   Topic1  -5.9537   0.1500
684               turn   794.265924   813.230163   Topic1  -4.9721   0.14

### Average Model Parameters

In [15]:
# Topics range
min_topics = num_topics - 5
max_topics = num_topics + 5
if min_topics <= 2:
    min_topics = 2
    max_topics = 10
step_size = 2
topics_range = range(min_topics, max_topics, step_size)

# List of alpha parameters
alpha_lst = ['auto']

# List of beta parameters
beta_lst = list(np.arange(beta - 0.9, beta + 1.3, .3))

avg_beta = 0.0
avg_num_topics = 0
avg_cv = 0.0
num_runs = 50

for run in range(num_runs):
    # Call to the method above
    alpha, beta, num_topics, max_cv = _get_best_num_topics(topics_range, alpha_lst, beta_lst, corpus)
    avg_beta += beta
    avg_num_topics += num_topics
    avg_cv += max_cv
    
avg_beta /= num_runs
avg_num_topics /= num_runs
avg_cv /= num_runs

print("Avg beta: ", avg_beta)
print("Avg Number of topics: ", avg_num_topics)
print("Avg Optimal Coherence Value: ", avg_cv)

100%|██████████| 1/1 [10:07<00:00, 607.86s/it]

Avg beta:  2.2760000000000002
Avg Number of topics:  7.84
Avg Optimal Coherence Value:  0.6265967004836291


### Run Example Models Using Average Parameters

In [18]:
for run in range(1):
    print("-------------------------------------------Run Number " + str(run) + "---------------------------------------------------------")
    print()
    ### Final Model Training

    lda_model = gensim.models.LdaModel(corpus=corpus,
                                          id2word=id2word,
                                          num_topics=8,
                                          chunksize=100,
                                          passes=10,
                                          alpha='auto',
                                          eta=2.28)
    coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
    print("Coherence: " + str(coherence_model_lda.get_coherence()))
    print()
    
    # Print the Keyword in the 20 topics
    pprint(lda_model.print_topics(num_topics=-1, num_words=20))
    doc_lda = lda_model[corpus]

    # Visualize the topics
    pyLDAvis.enable_notebook()
    LDAvis_prepared = gensimvis.prepare(lda_model, corpus, id2word, mds='mmds')
    pyLDAvis.save_html(LDAvis_prepared, "LDAvis_BBE_B_" + str(run) + ".html")
    print()
    print()

-------------------------------------------Run Number 0---------------------------------------------------------

Coherence: 0.3617882287216702

[(0,
  '0.013*"priest" + 0.013*"offer" + 0.012*"family" + 0.011*"tent" + '
  '0.010*"order" + 0.008*"death" + 0.008*"place" + 0.008*"number" + '
  '0.008*"unclean" + 0.008*"town" + 0.006*"altar" + 0.006*"levite" + '
  '0.006*"year" + 0.006*"live" + 0.006*"sin" + 0.005*"meeting" + 0.005*"work" '
  '+ 0.005*"fire" + 0.005*"burn" + 0.005*"burned_offere"'),
 (1,
  '0.003*"side" + 0.002*"doorway" + 0.002*"long" + 0.002*"cubit" + '
  '0.002*"cover" + 0.002*"measure" + 0.002*"work" + 0.001*"wheel" + '
  '0.001*"sword" + 0.001*"face" + 0.001*"servant" + 0.001*"eye" + '
  '0.001*"death" + 0.001*"stretch" + 0.001*"east_side" + 0.001*"water" + '
  '0.001*"food" + 0.001*"fear" + 0.001*"winged" + 0.001*"gold"'),
 (2,
  '0.000*"death" + 0.000*"place" + 0.000*"fear" + 0.000*"work" + 0.000*"town" '
  '+ 0.000*"word" + 0.000*"knowledge" + 0.000*"live" + 0.000*

C:\Users\15138\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:228: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info  = pd.DataFrame({'saliency': saliency, 'Term': vocab, \


## Chapters

### Preprocessing

In [19]:
# Create list of chapter documents and put them in a dataframe
chapterList = []
indexList = []
for chapter in bbe.groupby(['b','c'])['t']:
    chapterList.append(' '.join(chapter[1].tolist()))
    indexList.append(chapter[0])
chapters = pd.DataFrame({'b,c': indexList, 't': chapterList})

In [20]:
# Remove punctuation
chapters['t_processed'] = chapters['t'].map(lambda x: re.sub('[,\.!?]', '', x))

# Convert the titles to lowercase
chapters['t_processed'] = chapters['t_processed'].map(lambda x: x.lower())

#### Tokenize words and further clean-up text

data = chapters.t_processed.values.tolist()
data_words = list(sent_to_words(data))

#### Creating Bigram and Trigram Models

# Build the bigram and trigram models

bigram = gensim.models.Phrases(data_words, min_count=3, threshold=5)
trigram = gensim.models.Phrases(bigram[data_words], threshold=5)  
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

#### Remove Stopwords, Make Bigrams and Lemmatize

#list of stopwords
with open('stop_words_lst.txt') as f:
    standard_stop_words = f.read().splitlines()
with open('bible_stop_words_lst.txt') as f:
    bible_stop_words = f.read().splitlines()
stop_words = standard_stop_words + bible_stop_words

### Call the functions above

# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
nlp = spacy.load("en_core_web_lg", disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

### Data transformation: Corpus and Dictionary

# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

<>:2: DeprecationWarning: invalid escape sequence \.
<>:2: DeprecationWarning: invalid escape sequence \.
C:\Users\15138\AppData\Local\Temp/ipykernel_13700/92508175.py:2: DeprecationWarning: invalid escape sequence \.
  chapters['t_processed'] = chapters['t'].map(lambda x: re.sub('[,\.!?]', '', x))


### Model Selection

In [10]:
# Topics range
min_topics = 2
max_topics = 66
step_size = 2
topics_range = range(min_topics, max_topics, step_size)

# List of alpha parameters
alpha_lst = ['auto']

# List of beta parameters
beta_lst = list(np.arange(0.01, 3, 0.1))

# Call to the method above
alpha, beta, num_topics, max_cv = _get_best_num_topics(topics_range, alpha_lst, beta_lst, corpus)

print("alpha: ", alpha)
print("beta: ", beta)
print("Number of topics: ", num_topics)
print("Optimal Coherence Value: ", max_cv)

### Final Model Training

lda_model = gensim.models.LdaModel(corpus=corpus,
                                      id2word=id2word,
                                      num_topics=num_topics,
                                      chunksize=100,
                                      passes=10,
                                      alpha=alpha,
                                      eta=beta)

# Print the Keyword in the 10 topics
pprint(lda_model.print_topics(num_topics=-1, num_words=20))
doc_lda = lda_model[corpus]

# Visualize the topics
pyLDAvis.enable_notebook()
LDAvis_prepared = gensimvis.prepare(lda_model, corpus, id2word, mds='mmds')
LDAvis_prepared

100%|██████████| 1/1 [5:06:45<00:00, 18405.58s/it]


alpha:  auto
beta:  2.91
Number of topics:  28
Optimal Coherence Value:  0.8105564575355849
[(0,
  '0.000*"brothers_twelve" + 0.000*"corded_instrument" + 0.000*"jeremoth" + '
  '0.000*"shubael" + 0.000*"mahazioth" + 0.000*"sixteenth" + '
  '0.000*"eighteenth" + 0.000*"hothir" + 0.000*"izri" + 0.000*"ezer" + '
  '0.000*"eliathah" + 0.000*"bukkiah" + 0.000*"jesharelah" + 0.000*"giddalti" '
  '+ 0.000*"azarel" + 0.000*"twenty_fourth" + 0.000*"nineteenth" + '
  '0.000*"seventeenth" + 0.000*"learner" + 0.000*"thirteenth"'),
 (1,
  '0.000*"hammuel" + 0.000*"shuhah" + 0.000*"tochen" + 0.000*"tolad" + '
  '0.000*"potter" + 0.000*"shimri" + 0.000*"helah" + 0.000*"eliashib" + '
  '0.000*"ham" + 0.000*"ezem" + 0.000*"cozeba" + 0.000*"chelub" + '
  '0.000*"baalath" + 0.000*"ashhur" + 0.000*"ashbea" + 0.000*"aharhel" + '
  '0.000*"nogah" + 0.000*"jabez" + 0.000*"arnan" + 0.000*"mishma"'),
 (2,
  '0.000*"hammuel" + 0.000*"shuhah" + 0.000*"tochen" + 0.000*"tolad" + '
  '0.000*"potter" + 0.000*"shimri

C:\Users\15138\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:228: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info  = pd.DataFrame({'saliency': saliency, 'Term': vocab, \


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
12     0.098436 -0.276809       1        1  85.852332
13     0.080491 -0.064314       2        1  10.593424
15     0.021604  0.016534       3        1   2.981151
27    -0.006665  0.014047       4        1   0.366661
26    -0.008013  0.013148       5        1   0.078123
14    -0.008062  0.012942       6        1   0.006286
22    -0.008084  0.012928       7        1   0.006082
19    -0.008088  0.012921       8        1   0.005906
3     -0.008088  0.012920       9        1   0.005821
21    -0.008084  0.012928      10        1   0.005797
2     -0.008088  0.012921      11        1   0.005781
18    -0.008083  0.012931      12        1   0.005641
0     -0.008082  0.012932      13        1   0.005560
9     -0.008083  0.012929      14        1   0.005559
17    -0.008067  0.012931      15        1   0.005557
24    -0.008085  0.012939      16        1   0.005557
25    -0.008073  0.012938      17        1   0.005464
5     -0.008084  0.012929      18        1   0.005461
6     -0.008076  0.012937      19        1   0.005454
10    -0.008084  0.012929      20        1   0.005448
11    -0.008086  0.012926      21        1   0.005428
23    -0.008084  0.012928      22        1   0.005420
4     -0.008073  0.012937      23        1   0.005417
8     -0.008089  0.012918      24        1   0.005402
20    -0.008081  0.012932      25        1   0.005372
16    -0.008076  0.012937      26        1   0.005354
1     -0.008065  0.012940      27        1   0.005294
7     -0.008088  0.012922      28        1   0.005248, topic_info=                   Term         Freq        Total Category  logprob  loglift
73                 evil  1297.000000  1297.000000  Default  30.0000  30.0000
358                word  1066.000000  1066.000000  Default  29.0000  29.0000
201               heart   994.000000   994.000000  Default  28.0000  28.0000
84            knowledge  1028.000000  1028.000000  Default  27.0000  27.0000
115                fear   945.000000   945.000000  Default  26.0000  26.0000
13                earth   857.000000   857.000000  Default  25.0000  25.0000
362               faith  1037.000000  1037.000000  Default  24.0000  24.0000
376                 ear   827.000000   827.000000  Default  23.0000  23.0000
85                 life   777.000000   777.000000  Default  22.0000  22.0000
138                turn   748.000000   748.000000  Default  21.0000  21.0000
70                death  1115.000000  1115.000000  Default  20.0000  20.0000
208                time  1004.000000  1004.000000  Default  19.0000  19.0000
450                love   900.000000   900.000000  Default  18.0000  18.0000
43                place  1118.000000  1118.000000  Default  17.0000  17.0000
74                  eye   686.000000   686.000000  Default  16.0000  16.0000
25                 full   725.000000   725.000000  Default  15.0000  15.0000
430               power   666.000000   666.000000  Default  14.0000  14.0000
113              desire   770.000000   770.000000  Default  13.0000  13.0000
21                 food   808.000000   808.000000  Default  12.0000  12.0000
266             servant   850.000000   850.000000  Default  11.0000  11.0000
32                 live   840.000000   840.000000  Default  10.0000  10.0000
295               clear   645.000000   645.000000  Default   9.0000   9.0000
509                true   771.000000   771.000000  Default   8.0000   8.0000
344                king   613.000000   613.000000  Default   7.0000   7.0000
118                hand   661.000000   661.000000  Default   6.0000   6.0000
104                work   962.000000   962.000000  Default   5.0000   5.0000
173                 sin   777.000000   777.000000  Default   4.0000   4.0000
315             trouble   529.000000   529.000000  Default   3.0000   3.0000
75               father   687.000000   687.000000  Default   2.0000   2.0000
194         destruction   5

### Average Model Parameters

In [11]:
# Topics range
min_topics = num_topics - 5
max_topics = num_topics + 5
if min_topics <= 2:
    min_topics = 2
    max_topics = 10
step_size = 2
topics_range = range(min_topics, max_topics, step_size)

# List of alpha parameters
alpha_lst = ['auto']

# List of beta parameters
beta_lst = list(np.arange(beta - 0.9, beta + 1.3, .3))

avg_beta = 0.0
avg_num_topics = 0
avg_cv = 0.0
num_runs = 50

for run in range(num_runs):
    # Call to the method above
    alpha, beta, num_topics, max_cv = _get_best_num_topics(topics_range, alpha_lst, beta_lst, corpus)
    avg_beta += beta
    avg_num_topics += num_topics
    avg_cv += max_cv
    
avg_beta /= num_runs
avg_num_topics /= num_runs
avg_cv /= num_runs

print("Avg beta: ", avg_beta)
print("Avg Number of topics: ", avg_num_topics)
print("Avg Optimal Coherence Value: ", avg_cv)

100%|██████████| 1/1 [10:23<00:00, 623.86s/it]

Avg beta:  3.468
Avg Number of topics:  26.92
Avg Optimal Coherence Value:  0.8196513221612436


### Run Example Models Using Average Parameters

In [21]:
for run in range(1):
    print("-------------------------------------------Run Number " + str(run) + "---------------------------------------------------------")
    print()
    ### Final Model Training

    lda_model = gensim.models.LdaModel(corpus=corpus,
                                          id2word=id2word,
                                          num_topics=27,
                                          chunksize=100,
                                          passes=10,
                                          alpha='auto',
                                          eta=3.47)
    coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
    print("Coherence: " + str(coherence_model_lda.get_coherence()))
    print()

   # Print the Keyword in the 20 topics
    pprint(lda_model.print_topics(num_topics=-1, num_words=20))
    doc_lda = lda_model[corpus]

    # Visualize the topics
    pyLDAvis.enable_notebook()
    LDAvis_prepared = gensimvis.prepare(lda_model, corpus, id2word, mds='mmds')
    pyLDAvis.save_html(LDAvis_prepared, "LDAvis_BBE_C_" + str(run) + ".html")
    print()
    print()

-------------------------------------------Run Number 0---------------------------------------------------------

Coherence: 0.714163486293039

[(0,
  '0.000*"hammuel" + 0.000*"shuhah" + 0.000*"tochen" + 0.000*"tolad" + '
  '0.000*"potter" + 0.000*"shimri" + 0.000*"helah" + 0.000*"eliashib" + '
  '0.000*"ham" + 0.000*"ezem" + 0.000*"cozeba" + 0.000*"chelub" + '
  '0.000*"baalath" + 0.000*"ashhur" + 0.000*"ashbea" + 0.000*"aharhel" + '
  '0.000*"nogah" + 0.000*"jabez" + 0.000*"arnan" + 0.000*"mishma"'),
 (1,
  '0.000*"hammuel" + 0.000*"shuhah" + 0.000*"tochen" + 0.000*"tolad" + '
  '0.000*"potter" + 0.000*"shimri" + 0.000*"helah" + 0.000*"eliashib" + '
  '0.000*"ham" + 0.000*"ezem" + 0.000*"cozeba" + 0.000*"chelub" + '
  '0.000*"baalath" + 0.000*"ashhur" + 0.000*"ashbea" + 0.000*"aharhel" + '
  '0.000*"nogah" + 0.000*"jabez" + 0.000*"arnan" + 0.000*"mishma"'),
 (2,
  '0.000*"ibzan" + 0.000*"ephraimite" + 0.000*"pirathon" + 0.000*"crossing" + '
  '0.000*"sibboleth" + 0.000*"shibboleth" +

C:\Users\15138\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:228: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info  = pd.DataFrame({'saliency': saliency, 'Term': vocab, \
